In [10]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload
from pathlib import Path

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.read_data import *
from src.data.fire_data import *
from src.data.vn_data import *
from src.data.weather_data import *
from src.gen_functions import *
from src.features.dataset import Dataset

import scrapy
from scrapy.crawler import CrawlerProcess
from twisted.internet import reactor
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup

import logging

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Scrape location of VN power plants using scrapy 

# Example

In [15]:
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [16]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.ret': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': '../data/hanoi/plants_info.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

In [17]:
runner = CrawlerRunner()   #from Scrapy docs
runner.crawl(QuotesSpider)   

<Deferred at 0x7fa3c0527250 current result: <twisted.python.failure.Failure builtins.NameError: Module '__main__' doesn't define any object named 'ret'>>

# Wikipedia Page on Coal Power Plants in Vietnam

In [15]:
class WikiSpider(scrapy.Spider):
    name = "wiki"
    
    custom_settings = {
#         'LOG_LEVEL': logging.WARNING,
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': '../data/hanoi/plants_info.json' }
    
     
    start_urls = ['https://en.wikipedia.org/wiki/List_of_power_stations_in_Vietnam#For_gas,_coal-fired_power_plants']
        
#     def start_requests(self):
#         print(self.start_urls)
#         super().start_requests()
    
    def parse(self, response):
#         print('hello')
        links = response.xpath('//*[@id="mw-content-text"]/div[1]/div[3]/div/ol/li/span[2]/cite[1]/a[1]/@href')
        print(links)
        idxs = np.arange(len(links)) + 1
        for i, link in zip(idxs, links):
            if 'sourcewatch.org' in link.get():
                yield scrapy.Request(link.get(), callback=self.re_direct, cb_kwargs=dict(ref=i))
                
    def re_direct(self, response, ref):
    
        link = response.xpath('//*[@id="softredirect"]/a/@href').get()
        print(ref, link)
        yield scrapy.Request(link, callback=self.parse_loc, cb_kwargs=dict(ref=ref))
    
    def parse_loc(self, response, ref):
        info = response.xpath('//*[@id="mw-content-text"]/div/ul[1]/li')
        plant_info = {'ref':str(ref)} 
        for sub_info in info:
            k = sub_info.xpath('b/text()').get()
            k = k.replace(':','')
            try: 
                content = sub_info.xpath('text()').get().lstrip()
            except:
                content = ''
            plant_info[k] = content 
        print(plant_info)
        yield plant_info

In [16]:
runner = CrawlerRunner()   #from Scrapy docs
runner.crawl(WikiSpider)
setup()

27 https://www.gem.wiki/Binh_Dinh_power_station
4 https://www.gem.wiki/An_Khanh_power_station
40 https://www.gem.wiki/Duc_Giang_-_Lao_Cai_Chemicals_power_station
38 https://www.gem.wiki/Tan_Thanh_power_station
29 https://www.gem.wiki/Dung_Quat_power_station
39 https://www.gem.wiki/Van_Phong_power_station
34 https://www.gem.wiki/Ninh_Binh_power_station
33 https://www.gem.wiki/Nam_Dinh_power_station
24 https://www.gem.wiki/Vinh_Tan_power_station
22 https://www.gem.wiki/Uong_Bi_power_station
21 https://www.gem.wiki/Son_Dong_power_station
16 https://www.gem.wiki/Nghi_Son_power_station
20 https://www.gem.wiki/Quang_Ninh_power_station
18 https://www.gem.wiki/Nong_Son_power_station
25 https://www.gem.wiki/Vung_Ang_power_station
{'ref': '27', 'Sponsor': '', 'Parent company': '', 'Location': 'Bình Định province, Vietnam', 'Coordinates': '14.079, 109.205 (approximate)', 'Status': 'Cancelled', 'Gross Capacity': '3,200 MW', 'Type': '', 'Projected in service': '2026-29', 'Coal Type': '', 'Coal Sour

{'ref': '14', 'Sponsor': '', 'Parent company': '', 'Location': 'Mông Dương ward, Cẩm Phả city, Quảng Ninh province, Vietnam', 'Coordinates': '21.07324, 107.34847 (exact)', 'Status': 'Operating (Mông Dương-2); Operating (Mông Dương-1)', 'Gross Capacity': '', 'Type': '', 'In service': 'Mar. 2015 (Mông Dương-1 Unit 1); May 2015 (Mông Dương-1 Unit 2); Mar. 2015 (Mông Dương-2 Unit 1); Q3/4 2015 (Mông Dương-2 Unit 2)', 'Coal Type': 'Anthracite', 'Coal Source': '', 'Source of financing': 'Mông Dương-1: Export-Import Bank of Korea 40%, '}


In [201]:
class WikiSpider(scrapy.Spider):
    name = "wiki"
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': '../data/hanoi/plants_info.json' }
    
     
    start_urls = ['https://en.wikipedia.org/wiki/List_of_power_stations_in_Vietnam#For_gas,_coal-fired_power_plants']
        
    
    def parse(self, response):
         
        links = response.xpath('//*[@id="mw-content-text"]/div[1]/div[3]/div/ol/li/span[2]/cite[1]/a[1]/@href')
        idxs = np.arange(len(links)) + 1
        for i, link in zip(idxs, links):
            if 'sourcewatch.org' in link.get():
                yield scrapy.Request(link.get(), callback=self.re_direct, cb_kwargs=dict(ref=i))
                
    def re_direct(self, response, ref):
    
        link = response.xpath('//*[@id="softredirect"]/a/@href').get()
        print(ref, link)
        yield scrapy.Request(link, callback=self.parse_loc, cb_kwargs=dict(ref=ref))
    
    def parse_loc(self, response, ref):
        info = response.xpath('//*[@id="mw-content-text"]/div/ul[1]/li')
        plant_info = {'ref':str(ref)} 
        for sub_info in info:
            k = sub_info.xpath('b/text()').get()
            k = k.replace(':','')
            try: 
                content = sub_info.xpath('text()').get().lstrip()
            except:
                content = ''
            plant_info[k] = content 
        print(plant_info)
        yield plant_info

In [202]:
runner = CrawlerRunner()   #from Scrapy docs
runner.crawl(WikiSpider) 

29 https://www.gem.wiki/Dung_Quat_power_station
40 https://www.gem.wiki/Duc_Giang_-_Lao_Cai_Chemicals_power_station
33 https://www.gem.wiki/Nam_Dinh_power_station
4 https://www.gem.wiki/An_Khanh_power_station
38 https://www.gem.wiki/Tan_Thanh_power_station
27 https://www.gem.wiki/Binh_Dinh_power_station
39 https://www.gem.wiki/Van_Phong_power_station
34 https://www.gem.wiki/Ninh_Binh_power_station
{'ref': '29', 'Sponsor': 'Sembcorp Utilities Vietnam', 'Parent company': '', 'Location': 'Bình Sơn district, Quảng Ngãi province, Vietnam', 'Coordinates': '15.35790, 108.80911 (approximate)', 'Status': 'Cancelled', 'Gross Capacity': '1,200 MW'}
{'ref': '40', 'Sponsor': 'Duc Giang - Lao Cai Chemicals Joint Stock Company', 'Parent company': '', 'Location': 'Duc Giang Lao Cai chemical plant, Tằng Loỏng town, Bảo Thắng district, Lào Cai province, Vietnam', 'Coordinates': '22.3043761, 104.1469831 (exact)', 'Status': 'Construction', 'Gross Capacity': '100 MW (2 x 50 MW)', 'Type': '', 'Projected in 

In [222]:
with open('../data/hanoi/plants_info.json') as f:
    coals = json.load(f)
    
coals = pd.DataFrame(coals)
coals.head(2)

,ref,Sponsor,Parent company,Location,Coordinates,Status,Gross Capacity,Type,Projected in service,Coal Type,Coal Source,Source of financing,In service,Owner,In Service
0,29,Sembcorp Utilities Vietnam,,"Bình Sơn district, Quảng Ngãi province, Vietnam","15.35790, 108.80911 (approximate)",Cancelled,"1,200 MW",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40,Duc Giang - Lao Cai Chemicals Joint Stock Company,,"Duc Giang Lao Cai chemical plant, Tằng Loỏng t...","22.3043761, 104.1469831 (exact)",Construction,100 MW (2 x 50 MW),,2019-20,,"Quang Ninh Coal (expected type 6) or China, by...","30% internal capital; 70% loan of VND1,400 bil...",NaN,NaN,NaN


In [29]:
coals['latitude']  = coals['Coordinates'].str.extract('(\d+.\d+)').astype(float)
coals['longitude'] = coals['Coordinates'].str.extract(', (\d+.\d+)').astype(float)

In [30]:
coals.to_csv('../data/hanoi/coat_plants.csv', index= False)

In [31]:
coals = pd.read_csv('../data/hanoi/coat_plants.csv')

coals.head()

,ref,Sponsor,Parent company,Location,Coordinates,Status,Gross Capacity,Type,Projected in service,Coal Type,Coal Source,Source of financing,In service,Owner,In Service,Source of financing US$284 million,Gross generating capacity (cancelled),latitude,longitude
0,34,Ninh Binh Thermal Power JSC,NaN,"Thanh Bình ward, Ninh Bình city, Ninh Bình pro...","20.25267, 105.98447 (exact)",Cancelled,330 MW,NaN,NaN,Anthracite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.252670,105.984470
1,4,An Khánh Thermo-Power Joint Stock Company,NaN,"An Khánh commune, Dai Tu district, Thái Nguyên...","21.596912, 105.766609 (exact)",Operating (An Khánh-1 Unit 1); Operating (An K...,100 MW (An Khánh-1 Units 1 & 2: 50 MW),NaN,NaN,NaN,"An Khanh mine, Vietnam",NaN,4/2015 (An Khánh-1 Unit 1); 8/2015 (An Khánh-1...,NaN,NaN,NaN,NaN,21.596912,105.766609
2,39,NaN,NaN,"Ninh Phước commune, Ninh Hòa township, Khánh H...","12.47704, 109.29244 (exact)",Construction (Vân Phong-1); Cancelled (Vân Pho...,Vân Phong-1: 2 x 660 MW; Vân Phong-2: 2 x 660 MW,NaN,Vân Phong-1 Unit 1: 2023; Vân Phong-1 Unit 2: ...,Bituminous,NaN,Vân Phong-1:,NaN,NaN,NaN,NaN,NaN,12.477040,109.292440
3,20,NaN,NaN,"Hà Khánh ward, Hạ Long city, Quảng Ninh provin...","21.01182, 107.12883 (exact)",Quang Ninh-1 and Quang Ninh-2: Operating; Quan...,Quang Ninh-1: 600 MW (2 x 300 MW); Quang Ninh-...,NaN,NaN,Anthracite,NaN,Quảng Ninh-1:,NaN,Quang Ninh Thermal Power Joint Stock Company,NaN,NaN,NaN,21.011820,107.128830
4,24,NaN,NaN,"Vĩnh Tân commune, Tuy Phong district, Bình Thu...","11.32082, 108.80630 (exact)",NaN,"6,225 MW (1,200 MW (Vĩnh Tân-1); 1,245 MW (Vĩn...",NaN,NaN,Vinh Tan 1 and 2: Domestic Anthracite; Vinh Ta...,PT Berau and Maintime (Indonesia) (Vĩnh Tân-1),NaN,NaN,NaN,NaN,NaN,NaN,11.320820,108.806300


## Other Power Plants

In [32]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_power_stations_in_Vietnam#For_gas,_coal-fired_power_plants')

In [33]:
label_list = ['coal','gas','solar', 'wind', 'biomass', 'hydroelectricity']

In [36]:
power_plants

,Station,Province,Capacity (MW),Commission date,Sponsor/Owner,Status,Note,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref,type,Ref,...,Sponsors,Source,Location,Capacity (MWp),Construction Start Date,COD date,Plant name,Operation time,Technology,Fuel type
0,An Khanh 1 (Khanh Hoa power station),Thai Nguyen,120 (2x60),2015,An Khanh Electricity JSC,Operating,NaN,[4] 1208/QĐ-TTg,coal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cam Pha Phase I-II,Quang Ninh,680 (2x340),2011,VINACOMIN,Operating,NaN,[5] 1208/QĐ-TTg,coal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cao Ngan,Thai Nguyen,115 (2x57.5),2006,VINACOMIN,Operating,NaN,"[6] 428/QĐ-TTg, BCĐQGĐ-VP",coal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dong Nai Formosa Unit 1-2,Dong Nai,300 (2x150),2004,Hung Nghiep Formosa,Operating,NaN,[7],coal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dong Nai Formosa Unit 3,Dong Nai,150,2018,Hung Nghiep Formosa,Operating,NaN,[8]428/QĐ-TTg,coal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,NaN,NaN,7.5,NaN,EB Can Tho Environmental Energy LLC & China Ev...,NaN,NaN,NaN,hydroelectricity,[230],...,NaN,NaN,Can Tho,NaN,NaN,NaN,Can Tho,2018.0,Combustion,MSW
5,NaN,NaN,2,NaN,Urban Environment Company (URENCO) & Hitachi Z...,NaN,NaN,NaN,hydroelectricity,[231],...,NaN,NaN,Hanoi,NaN,NaN,NaN,Nam Son,2017.0,Combustion,MSW
6,NaN,NaN,150,NaN,NaN,NaN,NaN,NaN,hydroelectricity,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Sugar mills,NaN,CHP,NaN
7,NaN,NaN,75,NaN,NaN,NaN,In construction,NaN,hydroelectricity,NaN,...,NaN,NaN,Hanoi,NaN,NaN,NaN,Soc Son,NaN,Combustion,MSW


In [34]:
power_plants = []
for df, label in zip(dfs, label_list): 
    df['type'] = label
    power_plants.append(df)
    
power_plants= pd.concat(power_plants)

In [35]:
power_plants['Status'] = power_plants['Status'].fillna(power_plants['Unnamed: 7'])
#power_plants['Capacity (MW)'] = power_plants['Capacity (MW)'].fillna(power_plants['Capacity (MWp)'])

KeyError: 'Unnamed: 7'

In [38]:
power_plants.to_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/hanoi/power_plants.csv',index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/hanoi/power_plants.csv'

In [53]:
power_plants[~power_plants['Unnamed: 7'].isna()] 

,Station,Province,Capacity (MW),Commission date,Sponsor/Owner,Status,Note,Ref,type,Location,Capacity (MWp),Construction Start Date,COD date,Source,Unnamed: 7,Plant name,Operation time,Technology,Fuel type,Commission time
9,Trung Nam Ninh Thuan phase 2,Ninh Thuan,NaN,NaN,Trungnam Wind Power JSC,Under Construction,"ENERCON turbines, Siemens Inverter",NaN,wind,"Loi Hai and Bac Phong communes, Thuan Bac dist...",64.0,NaN,NaN,[165][166],Under Construction,NaN,NaN,NaN,NaN,NaN
10,Dai Phong wind farm,Binh Thuan,NaN,NaN,AC Energy+Blue Circle,Under Construction,AC Energy accounts for over 62% of the economi...,NaN,wind,"Thien Nghiep commune, Mui Ne ward, Phan Thiet ...",40.0,NaN,NaN,[167],Under Construction,NaN,NaN,NaN,NaN,NaN
11,Bac Lieu phase III (VN version),Bac Lieu,NaN,NaN,Super Wind Energy Cong Ly 1 JSC,Under Construction,Super Wind Energy Cong Ly 1 JSC = Cong Ly + Th...,NaN,wind,"Vinh Trach Dong commune, Bac Lieu city",142.0,30/01/2018,NaN,[168],Under Construction,NaN,NaN,NaN,NaN,NaN
12,Cong Ly (3 phase) (VN version),Soc Trang,NaN,NaN,Super Wind Energy Cong Ly 1 JSC,Under Construction,15x2MW units.,NaN,wind,"Lai Hoa commune, Vinh Chau district, Soc Trang",90.0,30/01/2018,NaN,[169],Under Construction,NaN,NaN,NaN,NaN,NaN
13,Lac Hoa 1,Soc Trang,NaN,NaN,Hoang Son Energy,Under Construction,NaN,NaN,wind,"Lac Hoa commune, Vinh Chau district, Soc Trang",30.0,NaN,NaN,[170],Under Construction,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Dak Po wind farm,Gia Lai,NaN,NaN,Mirat Energy + HLP,Approved,NaN,NaN,wind,"Dak Po district, Gia Lai province",200.0,NaN,NaN,[192],Approved,NaN,NaN,NaN,NaN,NaN
68,Mang Yang wind farm,Gia Lai,NaN,NaN,Mirat Energy + HLP,Approved,NaN,NaN,wind,"Mang Yang district, Gia Lai province",200.0,NaN,NaN,[192],Approved,NaN,NaN,NaN,NaN,NaN
69,Thai Hoa,Binh Thuan,NaN,NaN,Pacific Energy - Binh Thuan JSC,Approved,Metmast installed,NaN,wind,"Hoa Thang commune, Bac Binh district, Binh Thu...",82.5,NaN,NaN,[193],Approved,NaN,NaN,NaN,NaN,NaN
70,Thai Phong,Binh Thuan,NaN,NaN,Pacific Energy - Binh Thuan JSC,Approved,Metmast installed,NaN,wind,"Chi Cong, Hoa Minh and Phong Phu communes, Tuy...",52.5,NaN,NaN,[193],Approved,NaN,NaN,NaN,NaN,NaN


In [47]:
power_plants.dropna(axis=1,how='all')

,Station,Province,Capacity (MW),Commission date,Sponsor/Owner,Status,Note,Ref,type,Location,Capacity (MWp),Construction Start Date,COD date,Source,Unnamed: 7,Plant name,Operation time,Technology,Fuel type,Commission time
0,An Khanh 1 (Khanh Hoa power station),Thai Nguyen,2x58,2015,An Khanh Electricity JSC,Operating,NaN,[4],coal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cam Pha Phase I-II,Quang Ninh,2x340,2011,VINACOMIN,Operating,NaN,[5],coal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cao Ngan,Thai Nguyen,2x57.5,2006,VINACOMIN,Operating,NaN,[6],coal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dong Nai Formosa Unit 1-2,Dong Nai,2x150,2004,Hung Nghiep Formosa,Operating,NaN,[7],coal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dong Nai Formosa Unit 3,Dong Nai,150,2018,Hung Nghiep Formosa,Operating,NaN,[8],coal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Hoa Binh expansion,NaN,2x240,NaN,EVN,NaN,Under construction or proposed,Report 58/BC-CBT annex row I.21,hydroelectricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023.0
102,Ialy expansion,NaN,2x180,NaN,EVN,NaN,Under construction or proposed,Report 58/BC-CBT annex row I.22,hydroelectricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023.0
103,Tri An expansion,NaN,200,NaN,EVN,NaN,Under construction or proposed,Report 58/BC-CBT annex row I.23,hydroelectricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.0
104,My Ly,NaN,250,NaN,IPP,NaN,Under construction or proposed,Report 58/BC-CBT annex row V.5,hydroelectricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024.0


In [46]:
power_plants[(power_plants['type']=='gas') & (power_plants['Status']=='Operating')] 

,Station,Province,Capacity (MW),Commission date,Sponsor/Owner,Status,Note,Ref,type,Location,Capacity (MWp),Construction Start Date,COD date,Source,Unnamed: 7,Plant name,Operation time,Technology,Fuel type,Commission time
0,Ca Mau 1&2 gas power plant,Ca Mau,2x750,2008,PetroVietnam Power Ca Mau,Operating,NaN,[45] and decision 125/QD-DTDL annex 3 row 22-23,gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nhon Trach 1 gas power plant,Dong Nai,450,2009,PetroVietnam Power Nhon Trach,Operating,NaN,[46],gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nhon Trach 2 Combined Cycle Gas Turbine Plant,Dong Nai,750,2011,PetroVietnam Power Nhon Trach 2,Operating,NaN,[47],gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Phu My 2.1,Vung Tau,477,1997,"Phu My Thermal Power Company, GENCO3",Operating,NaN,[48] and Decision 125/QD-DTDL annex 1 row 15,gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ba Ria,Vung Tau,340,1992-2002,"Ba Ria Thermal Power Company, GENCO 3",Operating,NaN,[49] and Decision 125/QD-DTDL annex 3 row 21,gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Phu My 2.1 extension,Vung Tau,468,1999,"Phu My Thermal Power Company, GENCO 3",Operating,NaN,[50] and Decision 125/QD-DTDL annex 1 row 15,gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Phu My 4,Vung Tau,477,2004,"Phu My Thermal Power Company, GENCO 3",Operating,NaN,[50] and Decision 125/QD-DTDL annex 1 row 16,gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,O Mon,Can Tho,2x330,2009-2015,EVNGENCO2,Operating,NaN,[51][52],gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Phu My 1,Vung Tau,1118,2001,"Phu My Thermal Power Company, GENCO 3",Operating,NaN,[53][50] and Decision 125/QD-DTDL annex 1 row 14,gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Phu My 3,Vung Tau,720,2004,"Phu My 3 Bot Power Company LTD, BP Holdings BV...",Operating,NaN,[54][55] and Decision 125/QD-DTDL annex 3 row 18,gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df2 = pd.read_html('https://en.wikipedia.org/wiki/List_of_gas_power_plants_in_Vietnam')[0]
df2.head()

,Power plants,Other names,Sponsors,Capacity (MW),Status,Province,Commission date,Source,Note
0,O Mon III Combined Cycle Gas Turbine Plant,NaN,EVN (ODA),1x750,pre-permit/permitted,Can Tho,2025,MOIT Report 58/BC-CBT annex row I.13,NaN
1,O Mon IV Combined Cycle Gas Turbine Plant,NaN,EVN,1x750,pre-permit/permitted,Can Tho,2023,MOIT Report 58/BC-CBT annex row I.14,NaN
2,Dung Quat I Combined Cycle Gas Turbine Plant,NaN,EVN,750,pre-permit/permitted,Quang Ngai,2024,MOIT Report 58/BC-CBT annex row I.19,NaN
3,Dung Quat III Combined Cycle Gas Turbine Plant,NaN,EVN,750,pre-permit/permitted,Quang Ngai,2025,MOIT Report 58/BC-CBT annex row I.20,NaN
4,Nhon Trach 3&4 Combined Cycle Gas Turbine Plant,NaN,PVN,2x750,pre-permit/permitted,Dong Nai,2023-2024,MOIT Report 58/BC-CBT annex row II.5,NaN


In [45]:
df2[df2['Status']=='operating'] 

,Power plants,Other names,Sponsors,Capacity (MW),Status,Province,Commission date,Source,Note
12,Ca Mau 1&2 gas power plant,NaN,PetroVietnam Power Ca Mau,2x750,operating,Ca Mau,2008,[6] and decision 125/QD-DTDL annex 3 row 22-23,NaN
13,Nhon Trach 1 gas power plant,NaN,PetroVietnam Power Nhon Trach,450,operating,Dong Nai,2009,[7],NaN
14,Nhon Trach 2 Combined Cycle Gas Turbine Plant,NaN,PetroVietnam Power Nhon Trach 2,750,operating,Dong Nai,2011,[8],NaN
16,Phu My 2.1,NaN,"Phu My Thermal Power Company, GENCO3",477,operating,Vung Tau,1997,[10] and Decision 125/QD-DTDL annex 1 row 15,NaN
17,Ba Ria,NaN,"Ba Ria Thermal Power Company, GENCO 3",340,operating,Vung Tau,1992-2002,[11] and Decision 125/QD-DTDL annex 3 row 21,NaN
18,Phu My 2.1 extension,NaN,"Phu My Thermal Power Company, GENCO 3",468,operating,Vung Tau,1999,[12] and Decision 125/QD-DTDL annex 1 row 15,NaN
19,Phu My 4,NaN,"Phu My Thermal Power Company, GENCO 3",477,operating,Vung Tau,2004,[12] and Decision 125/QD-DTDL annex 1 row 16,NaN
22,O Mon,NaN,EVNGENCO2,2x330,operating,Can Tho,2009-2015,[15][16],NaN
23,Phu My 1,NaN,"Phu My Thermal Power Company, GENCO 3",1118,operating,Vung Tau,2001,[12][17] and Decision 125/QD-DTDL annex 1 row 14,NaN
24,Phu My 3,NaN,"Phu My 3 Bot Power Company LTD, BP Holdings BV...",720,operating,Vung Tau,2004,[18][19] and Decision 125/QD-DTDL annex 3 row 18,NaN
